# Memory


In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.language_models import FakeListChatModel
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.messages import trim_messages, HumanMessage


class PrintOutputCallback(BaseCallbackHandler):
    def on_chat_model_start(self, serialized, messages, **kwargs) -> None:
        print(f"Amount of input messages: {len(messages)}")


sessions: dict[str, InMemoryChatMessageHistory] = {}
handler = PrintOutputCallback()
llm = FakeListChatModel(responses=["ai1", "ai2", "ai3"])


def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in sessions:
        sessions[session_id] = InMemoryChatMessageHistory()
    return sessions[session_id]


trimmer = trim_messages(
    max_tokens=1,
    strategy="last",
    token_counter=len,
    include_system=True,
    start_on="human",
)

raw_chain = trimmer | llm
chain = RunnableWithMessageHistory(raw_chain, get_session_history)

config = {"callbacks": [PrintOutputCallback()], "configurable": {"session_id": "1"}}
_ = chain.invoke([HumanMessage("Hi!")], config=config)

print(f"History length: {len(sessions['1'].messages)}")

_ = chain.invoke(
    [HumanMessage("How are you?")],
    config=config,
)
print(f"History length: {len(sessions['1'].messages)}")

Amount of input messages: 1
History length: 2
Amount of input messages: 1
History length: 4


In [2]:
sessions["1"].messages

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai1', additional_kwargs={}, response_metadata={}, id='run--01d70160-fe9a-475f-ad8e-21d0e9a6f17c-0'),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai2', additional_kwargs={}, response_metadata={}, id='run--5ec601ac-7b62-4c9e-8aad-7a47c20a9b33-0')]

In [3]:
trimmer.invoke(sessions["1"].messages)

[]

In [5]:
from langgraph.graph import MessageGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage


def test_node(state):
    print(f"State: {state}")
    print(f"History length= {len(state[:-1])}")
    return [AIMessage("Hello!")]


builder = MessageGraph()
builder.add_node("test_node", test_node)
builder.add_edge(START, "test_node")
builder.add_edge("test_node", END)
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [6]:
_ = graph.invoke(
    [HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}}
)
_ = graph.invoke(
    [HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-b"}}
)
_ = graph.invoke(
    [HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}}
)

State: [HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='5683cd9c-3743-4255-8eb3-aff539229289')]
History length= 0
State: [HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='2952015a-3880-4d52-b78d-ba9f844c5b22')]
History length= 0
State: [HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='5683cd9c-3743-4255-8eb3-aff539229289'), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='ceb53564-5dfa-4fcc-8909-18d627e77cc9'), HumanMessage(content='test', additional_kwargs={}, response_metadata={}, id='8ea84a6f-3322-43fc-9676-99eba760caf8')]
History length= 2


In [7]:
checkpoints = list(memory.list(config={"configurable": {"thread_id": "thread-a"}}))

for check_point in checkpoints:
    print(check_point.config["configurable"]["checkpoint_id"])

1f0713b3-cd89-6fac-8004-3cb4f856af77
1f0713b3-cd88-6b16-8003-752bc551984e
1f0713b3-cd87-6fa4-8002-073e8b0f805a
1f0713b3-cd81-6fdc-8001-67e86116a3f6
1f0713b3-cd7f-6d36-8000-783310f23100
1f0713b3-cd7d-6266-bfff-343143016866
